# Car&Bids Data Transformation

In [76]:
import json
import re
import pandas as pd
import os

## Read auction files
This is for development only

In [ ]:
raw_auctions_path = ""
processed_auctions_path = ""

auction_files = os.listdir(raw_auctions_path)
auction_files

['2025-03-12.json',
 '2025-03-20.json',
 '2024-12-05.json',
 '2025-03-11.json',
 '2023-08-24.json',
 '2024-11-11.json',
 '2025-05-31.json',
 '2024-08-01_chunk601_to_700.json',
 '2024-07-28_chunk301_to_400.json',
 '2023-09-19.json',
 '2023-08-08.json',
 '2024-11-20.json',
 '2025-05-09.json',
 '2024-02-26.json',
 '2023-06-19.json',
 '2024-07-07_chunk101_to_200c.json',
 '2024-03-19.json',
 '2024-02-14.json',
 '2024-10-14.json',
 '2025-01-28.json',
 '2024-08-04_chunk1_to_100.json',
 '2025-03-19.json',
 '2023-10-16.json',
 '2023-09-26.json',
 '2024-12-30.json',
 '2024-08-15.json',
 '2024-12-11.json',
 '2024-07-28_chunk901_to_1000.json',
 '2024-01-18.json',
 '2023-06-05.json',
 'auctions1-1000_1.json',
 'auctions_c500s_501-1000_1.json',
 '2023-04-03.json',
 '2024-07-24.json',
 '2023-07-31.json',
 '2025-01-30.json',
 '2024-07-05_chunk401_to_500b.json',
 '2024-07-05_chunk801_to_900b.json',
 '2024-07-28_chunk501_to_600.json',
 '2023-07-10.json',
 '2024-08-01_chunk1101_to_1200.json',
 '2024-01-0

In [ ]:
def convert_to_list_dicts(data) -> list:
    """
    Converts nested auction data to flat dictionaries, handling special list fields.
    
    Args:
        data: Input auction data (dict or list format)
            - If dict: {url: {auction_data}}
            - If list: [{auction_data}]
            
    Returns:
        list: Flattened auction records with consistent structure
    """
    
    def extract_list_field(field_data, list_key=None):
        """Extracts list from field data with flexible key handling"""
        if field_data is None:
            return []
        if isinstance(field_data, list):
            return field_data
        if isinstance(field_data, dict):
            if list_key:
                return field_data.get(list_key, [])
        return []

    def process_auction(auction, url=None):
        """Process individual auction record"""

        auction_stats = auction.get('auction_stats', {}) or {}
        auction_stats.setdefault('view_count', 0)
        auction_stats.setdefault('watcher_count', 0)
        
        auction_data = {
            "auction_url": url if url else auction.get('auction_url'),
            "auction_title": auction.get('auction_title'),
            "auction_subtitle": auction.get('auction_subtitle'),
            "dougs_take": auction.get('dougs_take'),
            "auction_highlights": extract_list_field(
                auction.get('auction_highlights'), 
                list_key='bullet_points'
            ),
            "services": extract_list_field(
                auction.get('services') or auction.get('service_history'),
                list_key='items'
            ),
            "auction_equipment": auction.get('auction_equipment'),
            "modifications": auction.get('modifications'),
            "known_flaws": auction.get('known_flaws'),
            "included_items": auction.get('included_items'),
            "ownership_history": auction.get('ownership_history'),
            "seller_notes": auction.get('seller_notes'),
            "auction_videos":auction.get('auction_videos',[]),
            **auction.get('auction_quick_facts', {}),
            **auction_stats,
        }
        return auction_data


    # Handle input types
    if isinstance(data, dict):
        return [process_auction(data[auction], auction) for auction in data]
    if isinstance(data, list):
        return [process_auction(auction) for auction in data]
    raise ValueError("Input must be dictionary or list")

In [171]:
def read_auction_file(filename):
    with open(f'{raw_auctions_path}/{filename}') as file:
        data = json.load(file)
    return data

In [172]:
auction_data = []
for file in auction_files:
    data = read_auction_file(file)
    auction_data.extend(convert_to_list_dicts(data))

auction_data[:5]

[{'auction_url': 'https://carsandbids.com/auctions/3Lj5agnZ/2019-rolls-royce-ghost',
  'auction_title': '2019 Rolls-Royce Ghost',
  'auction_subtitle': '~12,300 Miles, Twin-Turbo V12 Power, Western-Owned, Unmodified',
  'dougs_take': "The Rolls-Royce Ghost is, without a doubt, one of the most impressive luxury cars around. Not only does this example feature gorgeous styling and an opulent interior, it's also equipped with a powerful twin-turbocharged V12 that's good for 563 horsepower. Other factory equipment includes 21-inch alloy wheels, self-righting wheel center caps, self-leveling air suspension, heated and cooled front seats, heated rear seats, a Shooting Star headliner, 4-zone automatic climate control, and adaptive cruise control. This Ghost also touts low mileage, no modifications, and ownership in Western states since new, for added peace of mind. Plus, this Ghost comes with a clean, accident-free Carfax report, which always increases the appeal.",
  'auction_highlights': ['T

## Clean auction data

In [173]:
raw_df = pd.DataFrame(auction_data)
raw_df

,auction_url,auction_title,auction_subtitle,dougs_take,auction_highlights,services,auction_equipment,modifications,known_flaws,included_items,...,reserve_status,auction_status,highest_bid_value,auction_date,view_count,bid_count,bids,watcher_count,buyer_username,seller_username
0,https://carsandbids.com/auctions/3Lj5agnZ/2019...,2019 Rolls-Royce Ghost,"~12,300 Miles, Twin-Turbo V12 Power, Western-O...","The Rolls-Royce Ghost is, without a doubt, one...",[The odometer on this Ghost currently displays...,[2025: Engine oil and filter changed],"[21-inch alloy wheels, Self-righting wheel cen...",[],"[Scuffs on bottom of bumper, Some wear on fron...",[1 key],...,Reserve,"Reserve not met, bid to","$139,000","Mar 12, 2025 8:19 PM UTC","12,107",31,"[$139,000, $138,500, $138,000, $137,500, $137,...",0.0,NaN,NaN
1,https://carsandbids.com/auctions/36WZplWn/1983...,1983 Toyota Land Cruiser Fire Truck,"NO RESERVE ~10,700 Miles, Japanese-Market Fire...",Now THIS is really cool! The 60-Series Toyota ...,"[This is a right-hand drive, Japanese-market t...",[2024: Engine oil and filter changed],"[Vinyl and cloth upholstery, Manual windows]","[Fire-fighting equipment (lights, siren, PA sy...","[Some chips and scratches around the exterior,...","[2 keys, Service record, Red wheels with tires...",...,No reserve,Sold to,"$10,499","Mar 12, 2025 8:12 PM UTC","8,320",18,"[$10,499, $10,200, $10,099, $9,999, $9,600, $9...",0.0,NaN,NaN
2,https://carsandbids.com/auctions/KDyGoeqp/2012...,2012 BMW M3 Coupe Competition Package,"6-Speed Manual, V8 Power, Fox Red Interior, So...","The E92 BMW M3 is an exciting car, as it's the...",[This BMW is equipped with a 6-speed manual tr...,"[February 2025: Engine oil and filter changed,...",[Competition Package (19-inch Style 359M stagg...,"[Turner Motorsport Stage 2 93 octane tune, Mac...","[Chips, scuffs, and scratches around the exter...","[2 keys, Owner’s manuals, Service records, 1 l...",...,Reserve,Sold to,"$23,750","Mar 12, 2025 8:11 PM UTC","13,722",53,"[$23,750, $23,500, $23,000, $22,750, $22,500, ...",0.0,NaN,NaN
3,https://carsandbids.com/auctions/98XjGMOQ/2008...,2008 Porsche Cayman S,"NO RESERVE 6-Speed Manual, Sport Chrono Packag...","I personally love the 987 Porsche Cayman, as i...",[This coupe is equipped with the desirable 6-s...,[April 2022: Engine oil and filter changed],[Sport Chrono Package without PCM (sport drivi...,"[Borla exhaust system, Bilstein parts in the s...","[The seller states that cylinder #6 has ""bore ...","[1 key, Owner's manuals]",...,No reserve,Sold to,"$9,500","Mar 12, 2025 8:00 PM UTC","11,835",15,"[$9,500, $9,100, $9,000, $8,100, $8,000, $7,20...",0.0,NaN,NaN
4,https://carsandbids.com/auctions/9e60821A/2014...,2014 BMW M6 Convertible,"6-Speed Manual, Twin-Turbo V8, San Marino Blue...",The F12 BMW M6 Convertible is a thrilling blen...,[This M6 features the desirable 6-speed manual...,"[August 2024 (49,645 miles): Wiper(s) replaced...","[Executive Package (heated steering wheel, act...",[],"[Rock chips on the front end, Scuffs on the fr...","[2 keys, Owner's manual]",...,Reserve,Sold to,"$42,000","Mar 12, 2025 7:59 PM UTC","12,166",39,"[$42,000, $41,000, $40,500, $40,000, $39,750, ...",0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29602,https://carsandbids.com/auctions/rGBw6LXE/2019...,2019 BMW 750i xDrive,"Highly Optioned, M Sport Package, V8 Power, Mo...",The 750i xDrive is a LOT of car for the money ...,[The attached Carfax history report shows no a...,"[August 2022 (31,037 miles): Engine oil and fi...",[Luxury Rear Seating Package (rear ventilated ...,[Tinted windows],[Some chips on hood and front bumper (pictured...,[2 keys],...,Reserve,Sold to,"$59,000","Sep 26, 2022 11:37 PM","9,136",32,"[$59,000, $58,000, $57,000, $56,000, $55,000, ...",0.0,NaN,NaN
29603,https://carsandbids.com/auctions/3Lo0XjaX/2000...,2000 Mercedes-Benz SL500,"NO RESERVE 1 Owner Until 2022, California-Owne...","The R129 SL is a modern classic Mercedes-

### rename column names

In [174]:
raw_df.columns = raw_df.columns.str.lower().str.replace(" ","_")
raw_df.columns

Index(['auction_url', 'auction_title', 'auction_subtitle', 'dougs_take',
       'auction_highlights', 'services', 'auction_equipment', 'modifications',
       'known_flaws', 'included_items', 'ownership_history', 'seller_notes',
       'auction_videos', 'make', 'model', 'mileage', 'vin', 'title_status',
       'location', 'seller', 'engine', 'drivetrain', 'transmission',
       'body_style', 'exterior_color', 'interior_color', 'seller_type',
       'reserve_status', 'auction_status', 'highest_bid_value', 'auction_date',
       'view_count', 'bid_count', 'bids', 'watcher_count', 'buyer_username',
       'seller_username'],
      dtype='object')

### collect urls of empty auctions

In [175]:
def get_and_remove_invalid_auctions(df):
    """
    Identifies auctions with invalid auction_status values by checking for key phrases.
    Removes them from the DataFrame and returns their URLs for re-scraping.
    
    Parameters:
        df (pd.DataFrame): The auction DataFrame.
    
    Returns:
        tuple: (cleaned DataFrame, list of auction URLs with invalid statuses)
    """
    # Check for key status indicators (case-insensitive)
    valid_status_mask = (
        df['auction_status'].str.lower().str.contains('sold|reserve not met|cancelled', na=False)
    )
    
    # Get URLs of invalid auctions
    rescrape_urls = df.loc[~valid_status_mask, 'auction_url'].tolist()
    
    # Keep only valid auctions
    clean_auctions_df = df.loc[valid_status_mask].copy()
    
    return clean_auctions_df, rescrape_urls

In [156]:
df, missing_urls = get_and_remove_invalid_auctions(raw_df)

### Convert 'auction_date' to datetime and sort df by auction_date desc


In [176]:
df['auction_date'] = pd.to_datetime(df['auction_date'], utc=True)
df = df.sort_values('auction_date', ascending=False).reset_index(drop=True)

### extract auction id

In [177]:
def extract_auction_id(url:str)->str:
    return url.strip().split("/")[4]

df['auction_id'] = df['auction_url'].apply(extract_auction_id)

### drop duplicates based on auction id

In [178]:
df = df.drop_duplicates('auction_id', keep='first')

### clean 'model' - remove '\nSave'

In [179]:
df['model'] = df['model'].str.split('\n').str[0].str.strip()

### Convert 'mileage' to integer

In [180]:
import re

def extract_mileage(value):
    if pd.isna(value):
        return None
    match = re.search(r'[\d,]+',value)
    if match:
        return int(match.group(0).replace(',',''))
    return None
df['mileage'] = df['mileage'].apply(extract_mileage) 

### convert 'highest-bid_value' to float

In [181]:
df['highest_bid_value'] = df['highest_bid_value'].str.replace('$','').str.replace(',','').astype(float)
df['highest_bid_value']

0        41500.0
1        24000.0
2         6300.0
3        17750.0
4        58500.0
          ...   
27503    15000.0
27504    21250.0
27505    32000.0
27506    31500.0
27507    35000.0
Name: highest_bid_value, Length: 27013, dtype: float64

### Convert 'bid_count' to integer

In [182]:
df['bid_count'] = pd.to_numeric(df['bid_count'], errors='coerce')

### Convert 'view_count' to integer

In [183]:
df['view_count'] = df['view_count'].astype(str).str.replace(',', '', regex=False)
df['view_count'] = pd.to_numeric(df['view_count'], errors='coerce').fillna(0).astype(int)

### Convert 'watcher_count' to integer

In [184]:
df['watcher_count'] = df['watcher_count'].astype('str').replace(',','', regex=True)
df['watcher_count'] = pd.to_numeric(df['watcher_count'], errors="coerce").fillna(0).astype(int)

### clean 'auction status' - change 'sold to' to 'sold'

In [185]:
df['auction_status'] = df['auction_status'].str.replace('Sold to','Sold').replace('Reserve not met, bid to', 'Reserve not met')

### Create boolean col for reserve status

In [186]:
df['reserve_met'] = df['auction_status'].str.lower().eq('sold')

### remove 'follow' from seller

In [187]:
df['seller'] = df['seller'].str.split('\n').str[0].str.strip()

### clean bids

In [188]:
def clean_bids(bids_list):
    try:
        return [int(bid.replace("$",'').replace(',','')) for bid in bids_list]
    except Exception as e:
        return []
    
df['bids'] = df['bids'].apply(clean_bids)

### Split 'title_status' into 'title_status_clean' and 'title_state'

In [189]:
df['title_status_cleaned'] = df['title_status'].str.extract(r'^(.*?) \(')
df['title_state'] = df['title_status'].str.extract(r'\((.*?)\)')

### Split 'location' into 'city' and 'state'

In [190]:
def extract_city_state(location):
        if pd.isna(location):
            return None, None
        try:
            parts = location.rsplit(",", 1)
            if len(parts) == 2:
                city = parts[0].strip()
                state = parts[1].strip().split(" ")[0]
                return city,state
            else:
                return parts[0].strip(), None
        except Exception:
            raise
df[['city','state']] = df['location'].apply(extract_city_state).apply(pd.Series)

### clean transmission

In [191]:
def clean_transmission(trans_str):
    if not trans_str or not isinstance(trans_str, str):
        return None, None

    trans_str = trans_str.lower()
    transmission_type = 'Other'
    if 'manual' in trans_str:
        transmission_type = 'Manual' 
    elif 'auto' in trans_str:
        transmission_type = 'Automatic'

    match = re.search(r'(\d+)-speed', trans_str)
    gears = int(match.group(1)) if match else None

    return transmission_type, gears

df['transmission_type'], df['gears'] = zip(*df['transmission'].map(clean_transmission))
df['transmission_type'] = df['transmission_type'].astype('object') 

### clean drivetrain

In [192]:
def clean_drivetrain(drive_str):
    if not drive_str or not isinstance(drive_str, str):
        return 'Other'

    drive_str = drive_str.lower()

    if '4wd' in drive_str and 'awd' in drive_str:
        return '4WD/AWD'
    elif 'front' in drive_str:
        return 'FWD'
    elif 'rear' in drive_str:
        return 'RWD'
    elif 'awd' in drive_str or 'all-wheel' in drive_str:
        return 'AWD'
    elif '4wd' in drive_str or 'four-wheel' in drive_str:
        return '4WD'
    else:
        return 'Other'
    
df['drivetrain'] = df['drivetrain'].apply(clean_drivetrain)

 ### extract bids features

In [193]:
import numpy as np

def extract_bid_features(bids_list):
    if not bids_list or not isinstance(bids_list, list) or len(bids_list) < 2:
        return pd.Series(
            {
                'max_bid': np.nan,
                'min_bid': np.nan,
                'mean_bid': np.nan,
                'median_bid': np.nan,
                'bid_range': np.nan,
            }
        )

    return pd.Series(
        {
            'max_bid': max(bids_list),
            'min_bid': min(bids_list),
            'mean_bid': np.mean(bids_list),
            'median_bid': np.median(bids_list),
            'bid_range': max(bids_list) - min(bids_list),
        }
    )


features_df = df['bids'].apply(extract_bid_features)
df = df.join(features_df)

### add count fields for auction flaws, services, equipment, extra items, highlights, 

In [194]:
def count_list(x):
    return len(x) if isinstance(x, list) else None

df['highlight_count'] = df['auction_highlights'].apply(count_list)
df['equipment_count'] = df['auction_equipment'].apply(count_list)
df['mod_count'] = df['modifications'].apply(count_list)
df['flaw_count'] = df['known_flaws'].apply(count_list)
df['service_count'] = df['services'].apply(count_list)
df['included_items_count'] = df['included_items'].apply(count_list)
df['video_count'] = df['auction_videos'].apply(count_list)

In [195]:
def extract_manufacture_year(url):
    if pd.isna(url):
        return None
    try:
        return int(url.strip().split("/")[-1].split("-")[0])
    except Exception:
        None

df['manufacture_year'] = df['auction_url'].apply(extract_manufacture_year)

In [196]:
df.columns

Index(['auction_url', 'auction_title', 'auction_subtitle', 'dougs_take',
       'auction_highlights', 'services', 'auction_equipment', 'modifications',
       'known_flaws', 'included_items', 'ownership_history', 'seller_notes',
       'auction_videos', 'make', 'model', 'mileage', 'vin', 'title_status',
       'location', 'seller', 'engine', 'drivetrain', 'transmission',
       'body_style', 'exterior_color', 'interior_color', 'seller_type',
       'reserve_status', 'auction_status', 'highest_bid_value', 'auction_date',
       'view_count', 'bid_count', 'bids', 'watcher_count', 'buyer_username',
       'seller_username', 'auction_id', 'auction_saving_date', 'reserve_met',
       'title_status_cleaned', 'title_state', 'city', 'state',
       'transmission_type', 'gears', 'max_bid', 'min_bid', 'mean_bid',
       'median_bid', 'bid_range', 'highlight_count', 'equipment_count',
       'mod_count', 'flaw_count', 'service_count', 'included_items_count',
       'video_count', 'manufacture_yea

In [197]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27013 entries, 0 to 27507
Data columns (total 59 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   auction_url           27013 non-null  object             
 1   auction_title         26983 non-null  object             
 2   auction_subtitle      26964 non-null  object             
 3   dougs_take            24391 non-null  object             
 4   auction_highlights    27013 non-null  object             
 5   services              27013 non-null  object             
 6   auction_equipment     25661 non-null  object             
 7   modifications         25661 non-null  object             
 8   known_flaws           27013 non-null  object             
 9   included_items        27013 non-null  object             
 10  ownership_history     26892 non-null  object             
 11  seller_notes          1352 non-null   object             
 12  auction_v

In [198]:
df[['auction_id','auction_date']]

,auction_id,auction_date
0,3646O08y,2025-06-12 20:15:00+00:00
1,rMgWezp8,2025-06-12 20:10:00+00:00
2,rwA8RZWM,2025-06-12 20:09:00+00:00
3,r4Ga6koA,2025-06-12 20:05:00+00:00
4,rbpNxPjL,2025-06-12 19:57:00+00:00
...,...,...
27503,KVGJe7BP,2020-06-16 20:06:00+00:00
27504,9elpnQDd,2020-06-16 19:00:00+00:00
27505,9nbW8GPJ,2020-06-15 21:02:00+00:00
27506,rJnm14vg,2020-06-15 20:04:00+00:00


## Save files

In [109]:
## one json file

df.to_json('test_data.json', orient='records',lines=True)

In [201]:
## group by auction data then save to json

df['auction_saving_date'] = pd.to_datetime(df['auction_date']).dt.date
for day,group in df.groupby('auction_saving_date'):
    group.drop(columns=['auction_saving_date']).to_json(f'{processed_auctions_path}/{day}.json', orient='records', lines=True)

In [108]:
## one csv file
df.to_csv('test_data.csv', index=False)